# **Expected Goals Classifier**

## Overview

Create an Expected Goals (xG) classification model using existing historical match data to produce actionable recommendations which can be utilized in technical and tactical analysis to improve goal-scoring.

Project detailed on Github: [Expected Goals Classifier]()

# Data Cleaning Notebook

Continued from expected_goals_data_extraction_notebook

*Notebook 2 of 7*

### Index

1. Data extracted in [expected_goals_data_extraction_notebook]()
2. Data cleaned in [expected_goals_data_cleaning_notebook]()
3. Features engineered in [expected_goals_feature_engineering_notebook]()
4. Data explored in [expected_goals_data_exploration_notebook]()
5. Data preprocessed in [expected_goals_data_preprocessing_notebook]()
6. Predictions modeled in [expected_goals_model_fitting_notebook]()
7. Conclusions in [expected_goals_model_assessment_notebook]()

<a id = 'packages'></a>
# Packages

In [1]:
# Drive  and IO to access saved files
from google.colab import drive, files
drive.mount('/content/drive')

import io

# Pathlib for file retrieval and assessment
import pathlib
from pathlib import Path as path

# warnings to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Pandas for dataframes
import pandas as pd

# Numpy for mathematical functions
import numpy as np

# ProfileReport for exploratory data analysis
!pip install http://github.com/pandas-profiling/pandas-profiling/archive/master.zip
from pandas_profiling import ProfileReport as pr

# Matplotlib, and Plotly for visualizations
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px

# Scipy for statistical functions

from scipy import stats

Mounted at /content/drive
     |████████████████████████████████| 34.6 MB 7.6 kB/s 
     |████████████████████████████████| 10.1 MB 24.8 MB/s 
     |████████████████████████████████| 596 kB 46.5 MB/s 
     |████████████████████████████████| 102 kB 12.0 MB/s 
     |████████████████████████████████| 675 kB 34.4 MB/s 
     |████████████████████████████████| 4.7 MB 31.7 MB/s 
     |████████████████████████████████| 62 kB 782 kB/s 
     |████████████████████████████████| 812 kB 41.9 MB/s 
     |████████████████████████████████| 28.5 MB 49 kB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.1.1-py2.py3-none-any.whl size=261261 sha256=21da7cb255d1659bf1d030d9656a8e0d803ad04090c7838095f4519789e80d4c
  Stored in directory: /tmp/pip-ephem-wheel-cache-43bw4xtn/wheels/6f/98/05/b692e0ca1e33d89c0d1b5663fa20b18d015af6312c2a6a826d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=1269047714f198f539dc41736af2ca9242d3c6afbe79a0f1a1de6ccdd21fa164


# Data

Data sourced from [StatsBomb](https://statsbomb.com/), a United Kingdom based football (soccer) data analytics company.

StatsBomb have provided free access to their proprietary dataset via GitHub: [StatsBomb Open Data](https://github.com/statsbomb/open-data)

In [30]:
# Import extracted_data from expected_goals_data_extraction_notebook

extracted_data = pd.read_parquet('/content/drive/MyDrive/expected_goals/data_extraction/dataframes/extracted_data.parquet')

In [31]:
extracted_data.head()

,id,index_x,period_x,timestamp_x,minute_x,second_x,type_x,possession_x,possession_team_x,play_pattern_x,team_x,player_x,position_x,location_x,duration_x,under_pressure_x,related_events_x,match_id_x,shot_statsbomb_xg,shot_end_location,shot_key_pass_id,shot_technique,shot_outcome,shot_type,shot_body_part,shot_freeze_frame,shot_one_on_one,shot_aerial_won,shot_open_goal,shot_first_time,out_x,shot_redirect,shot_deflected,off_camera_x,shot_saved_off_target,shot_saved_to_post,shot_follows_dribble,index_y,period_y,timestamp_y,...,second_y,type_y,possession_y,possession_team_y,play_pattern_y,team_y,player_y,position_y,location_y,duration_y,related_events_y,match_id_y,pass_recipient,pass_length,pass_angle,pass_height,pass_end_location,pass_body_part,pass_type,under_pressure_y,pass_outcome,pass_aerial_won,pass_assisted_shot_id,pass_shot_assist,off_camera_y,pass_switch,pass_through_ball,pass_technique,pass_backheel,pass_cross,counterpress,pass_cut_back,pass_deflected,pass_goal_assist,pass_miscommunication,pass_inswinging,pass_straight,pass_outswinging,pass_no_touch,out_y
0,8f5a3b7c-db0b-42ec-bac0-adc0bedca2ea,258,1,00:04:38.609,4,38,Shot,11,Chelsea FCW,Regular Play,Chelsea FCW,Francesca Kirby,Center Forward,"[109.0, 46.0]",0.278800,True,"[011167bc-9cbc-46a3-9b7b-28065eab7af1, 2c37831...",19743,0.266154,"[112.0, 45.0]",bf82ea91-c3e3-4d8c-b91d-c9d0ccd44f11,Normal,Blocked,Open Play,Left Foot,"[{'location': [104.0, 50.0], 'player': {'id': ...",None,None,None,None,None,None,None,None,None,None,None,253.0,1.0,00:04:35.786,...,35.0,Pass,11.0,Chelsea FCW,Regular Play,Chelsea FCW,Bethany England,Left Midfield,"[95.0, 49.0]",1.361685,"[58da4d74-7684-405d-a8cc-bef1d658f1b6, 60d1337...",19743.0,Francesca Kirby,11.180340,0.463648,Ground Pass,"[105.0, 54.0]",Left Foot,None,True,None,None,8f5a3b7c-db0b-42ec-bac0-adc0bedca2ea,True,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,60ead7a6-4aa2-41ab-85a1-21357f50e4e0,542,1,00:11:45.046,11,45,Shot,24,Chelsea FCW,From Free Kick,Chelsea FCW,Bethany England,Left Midfield,"[113.0, 35.0]",0.256730,True,"[a4b77cbb-14d0-4bd3-ba8b-7312335098fe, b9b246c...",19743,0.093521,"[120.0, 32.9, 0.4]",b99082e1-812b-48dd-bf94-8856b1ff079b,Normal,Off T,Open Play,Head,"[{'location': [108.0, 45.0], 'player': {'id': ...",True,True,None,None,None,None,None,None,None,None,None,539.0,1.0,00:11:42.863,...,42.0,Pass,24.0,Chelsea FCW,From Free Kick,Chelsea FCW,Erin Cuthbert,Right Midfield,"[82.0, 54.0]",2.103800,[540a29f4-8533-4852-b492-307d124cf084],19743.0,Bethany England,37.735924,-0.558599,High Pass,"[114.0, 34.0]",Right Foot,Free Kick,None,None,None,60ead7a6-4aa2-41ab-85a1-21357f50e4e0,True,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,f68deb6f-0711-4b9d-8081-122dc3722c55,614,1,00:18:03.461,18,3,Shot,29,Chelsea FCW,Regular Play,Chelsea FCW,Drew Spence,Left Defensive Midfield,"[94.0, 43.0]",1.147883,True,"[3c03553f-3bed-4d21-8096-ed4ef269da62, bb13e23...",19743,0.036171,"[120.0, 42.8, 0.5]",5022d0b3-ea32-42a8-bd41-b46cc244beb9,Normal,Saved,Open Play,Left Foot,"[{'location': [118.0, 41.0], 'player': {'id': ...",None,None,None,None,None,None,None,None,None,None,None,610.0,1.0,00:18:01.596,...,1.0,Pass,29.0,Chelsea FCW,Regular Play,Chelsea FCW,So-yun Ji,Center Attacking Midfield,"[98.0, 60.0]",0.918187,"[753c6e78-72f9-4963-bcb7-c3e4ed58be6a, c884125...",19743.0,Drew Spence,11.180340,-2.034444,Ground Pass,"[93.0, 50.0]",Right Foot,None,True,None,None,f68deb6f-0711-4b9d-8081-122dc3722c55,True,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,f301190f-cc0a-4f16-8278-27e5279ea24e,877,1,00:23:11.935,23,11,Shot,43,Birmingham City WFC,From Goal Kick,Birmingham City WFC,Chloe Arthur,Right Back,"[86.0, 34.0]",2.161012,True,"[0bfe1b6c-d690-41a6-be3e-f9b6295ddd85, 570e15b...",19743,0.016625,"[119.0, 33.3, 0.5]",fdf4a564-4973-46e5-bc07-d84785f8c183,Normal,Off T,Open Play,Left Foot,"[{'location': [78.0, 58.0], 'player': {'id': 1...",None,None,None,

# ProfileReport

In [ ]:
pr_report = pr(extracted_data)
pr_report

Output hidden; open in https://colab.research.google.com to view.

# Target Feature

In [32]:
# The target feature is shot_outcome

In [33]:
# Display value counts for shot_outcome

extracted_data['shot_outcome'].value_counts(dropna = False)

Off T               1912
Saved               1531
Blocked             1460
Goal                 664
Wayward              336
Post                 136
Saved Off Target      24
Saved to Post         17
Name: shot_outcome, dtype: int64

In [34]:
# xG measures the likelihood a shot will result in a goal
# The target measurement needs to be if the shot was a goal

# Update shot_outcome to boolean goal

extracted_data['shot_outcome'] = extracted_data['shot_outcome'].apply(lambda x: 1 if x == 'Goal' else 0)

extracted_data.rename(columns = {'shot_outcome' : 'goal'},
                      inplace = True)

extracted_data['goal'] = extracted_data['goal'].astype(bool)

In [35]:
extracted_data['goal'].value_counts()

False    5416
True      664
Name: goal, dtype: int64

In [36]:
print("goal Percent 'True':",
      round((((sum(extracted_data['goal'])) /
              (len(extracted_data))) *
             100),
            2),
      '%')

goal Percent 'True': 10.92 %


# Irrelevant Data

In [37]:
print('Total Features:',
      extracted_data.shape[1])

Total Features: 81


In [38]:
# List current features

list(extracted_data.columns.values)

['id',
 'index_x',
 'period_x',
 'timestamp_x',
 'minute_x',
 'second_x',
 'type_x',
 'possession_x',
 'possession_team_x',
 'play_pattern_x',
 'team_x',
 'player_x',
 'position_x',
 'location_x',
 'duration_x',
 'under_pressure_x',
 'related_events_x',
 'match_id_x',
 'shot_statsbomb_xg',
 'shot_end_location',
 'shot_key_pass_id',
 'shot_technique',
 'goal',
 'shot_type',
 'shot_body_part',
 'shot_freeze_frame',
 'shot_one_on_one',
 'shot_aerial_won',
 'shot_open_goal',
 'shot_first_time',
 'out_x',
 'shot_redirect',
 'shot_deflected',
 'off_camera_x',
 'shot_saved_off_target',
 'shot_saved_to_post',
 'shot_follows_dribble',
 'index_y',
 'period_y',
 'timestamp_y',
 'minute_y',
 'second_y',
 'type_y',
 'possession_y',
 'possession_team_y',
 'play_pattern_y',
 'team_y',
 'player_y',
 'position_y',
 'location_y',
 'duration_y',
 'related_events_y',
 'match_id_y',
 'pass_recipient',
 'pass_length',
 'pass_angle',
 'pass_height',
 'pass_end_location',
 'pass_body_part',
 'pass_type',
 'un

### Duplicate Features

In [39]:
# Define duplicate features

duplicate_features = ['shot_saved_off_target',
                     'shot_saved_to_post',
                     'pass_outcome',
                     'pass_assisted_shot_id',
                     'pass_shot_assist',
                     'pass_goal_assist',
                     'pass_end_location',
                     'index_y',
                     'period_y',
                     'timestamp_y',
                     'minute_x',
                     'second_x',
                     'minute_y',
                     'second_y',
                     'type_y',
                     'possession_y',
                     'possession_team_y',
                     'play_pattern_y',
                     'team_y',
                     'player_y',
                     'position_y',
                     'location_y',
                     'duration_y',
                     'related_events_y',
                     'match_id_y',
                     'under_pressure_y',
                     'off_camera_y',
                     'out_y']

In [40]:
# Drop duplicate features

extracted_data.drop(duplicate_features,
                    axis = 1,
                    inplace = True)

### Non Shot-Specific Features

In [41]:
# Define features unrelated to shots

non_shot_specific_features = ['id',
                     'index_x',
                     'type_x',
                     'possession_x',
                     'possession_team_x',
                     'team_x',
                     'player_x',
                     'position_x',
                     'duration_x',
                     'related_events_x',
                     'match_id_x',
                     'shot_key_pass_id',
                     'shot_freeze_frame',
                     'out_x',
                     'off_camera_x',
                     'shot_aerial_won',
                     'pass_recipient',
                     'pass_body_part',
                     'pass_aerial_won',
                     'pass_deflected',
                     'pass_miscommunication',
                     'pass_no_touch']

In [42]:
# Drop features unrelated to shots

extracted_data.drop(non_shot_specific_features,
                    axis = 1,
                    inplace = True)

### Results

In [43]:
print('Total Features:',
      extracted_data.shape[1])

Total Features: 31


In [44]:
# List updated features

list(extracted_data.columns.values)

['period_x',
 'timestamp_x',
 'play_pattern_x',
 'location_x',
 'under_pressure_x',
 'shot_statsbomb_xg',
 'shot_end_location',
 'shot_technique',
 'goal',
 'shot_type',
 'shot_body_part',
 'shot_one_on_one',
 'shot_open_goal',
 'shot_first_time',
 'shot_redirect',
 'shot_deflected',
 'shot_follows_dribble',
 'pass_length',
 'pass_angle',
 'pass_height',
 'pass_type',
 'pass_switch',
 'pass_through_ball',
 'pass_technique',
 'pass_backheel',
 'pass_cross',
 'counterpress',
 'pass_cut_back',
 'pass_inswinging',
 'pass_straight',
 'pass_outswinging']

# Data Types

In [45]:
extracted_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 6079
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   period_x              6080 non-null   int64  
 1   timestamp_x           6080 non-null   object 
 2   play_pattern_x        6080 non-null   object 
 3   location_x            6080 non-null   object 
 4   under_pressure_x      1045 non-null   object 
 5   shot_statsbomb_xg     6080 non-null   float64
 6   shot_end_location     6080 non-null   object 
 7   shot_technique        6080 non-null   object 
 8   goal                  6080 non-null   bool   
 9   shot_type             6080 non-null   object 
 10  shot_body_part        6080 non-null   object 
 11  shot_one_on_one       330 non-null    object 
 12  shot_open_goal        70 non-null     object 
 13  shot_first_time       1290 non-null   object 
 14  shot_redirect         22 non-null     object 
 15  shot_deflected       

## Boolean Features

In [46]:
# Define boolean features

boolean_features = ['shot_one_on_one',
                    'shot_open_goal',
                    'shot_first_time',
                    'shot_redirect',
                    'shot_deflected',
                    'shot_follows_dribble',
                    'under_pressure_x',
                    'counterpress',
                    'pass_switch',
                    'pass_through_ball',
                    'pass_backheel',
                    'pass_cross',
                    'pass_cut_back',
                    'pass_inswinging',
                    'pass_straight',
                    'pass_outswinging']

In [47]:
# Convert boolean features to boolean data type

extracted_data[boolean_features] = extracted_data[boolean_features].astype(bool)

## Datetime

In [48]:
# Convert timestamp_x datatype to datetime

extracted_data['timestamp_x'] = extracted_data['timestamp_x'].astype(str)
extracted_data['timestamp_x'] = pd.to_datetime(extracted_data['timestamp_x'])

## Results

In [49]:
extracted_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 6079
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   period_x              6080 non-null   int64         
 1   timestamp_x           6080 non-null   datetime64[ns]
 2   play_pattern_x        6080 non-null   object        
 3   location_x            6080 non-null   object        
 4   under_pressure_x      6080 non-null   bool          
 5   shot_statsbomb_xg     6080 non-null   float64       
 6   shot_end_location     6080 non-null   object        
 7   shot_technique        6080 non-null   object        
 8   goal                  6080 non-null   bool          
 9   shot_type             6080 non-null   object        
 10  shot_body_part        6080 non-null   object        
 11  shot_one_on_one       6080 non-null   bool          
 12  shot_open_goal        6080 non-null   bool          
 13  shot_first_time   

# Missing Values

In [50]:
extracted_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 6079
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   period_x              6080 non-null   int64         
 1   timestamp_x           6080 non-null   datetime64[ns]
 2   play_pattern_x        6080 non-null   object        
 3   location_x            6080 non-null   object        
 4   under_pressure_x      6080 non-null   bool          
 5   shot_statsbomb_xg     6080 non-null   float64       
 6   shot_end_location     6080 non-null   object        
 7   shot_technique        6080 non-null   object        
 8   goal                  6080 non-null   bool          
 9   shot_type             6080 non-null   object        
 10  shot_body_part        6080 non-null   object        
 11  shot_one_on_one       6080 non-null   bool          
 12  shot_open_goal        6080 non-null   bool          
 13  shot_first_time   

## No Pass Shots

### Assess Shot Events Not Preceded by a Pass

In [51]:
print('pass_length NA:',
      (sum(extracted_data['pass_length'].isna())),
      '\n',
      'pass_angle NA:',
      (sum(extracted_data['pass_angle'].isna())),
      '\n',
      'pass_height NA:',
      (sum(extracted_data['pass_height'].isna())))

pass_length NA: 1942 
 pass_angle NA: 1942 
 pass_height NA: 1942


In [52]:
print('pass_length NA =/= pass_angle NA:',
      (1942 - (sum(extracted_data.loc[extracted_data['pass_length'].isna()]['pass_angle'].isna()))),
      '\n',
      'pass_length NA =/= pass_height NA:',
      (1942 - (sum(extracted_data.loc[extracted_data['pass_length'].isna()]['pass_height'].isna()))),
      '\n',
      'pass_angle NA =/= pass_height NA:',
      (1942 - (sum(extracted_data.loc[extracted_data['pass_angle'].isna()]['pass_height'].isna()))))

pass_length NA =/= pass_angle NA: 0 
 pass_length NA =/= pass_height NA: 0 
 pass_angle NA =/= pass_height NA: 0


In [53]:
# Note: pass_length, pass_angle, and pass_height each have 1942 missing values

# Note: The 1942 missing values for each pass_length, pass_angle, and pass_height
# are the same events

# Assume these missing values are shots which were not preceded by a pass ('No Pass Shots')

In [54]:
print('Percent no pass shots:',
      (round((((sum(extracted_data['pass_length'].isna())) /
               (len(extracted_data))) *
              100),
             2)),
      '%')

Percent no pass shots: 31.94 %


In [55]:
# 31.84% of values is too significant to drop

# Features are too significant to drop: No preceding pass is valuable in its
# description of the play preceeding the shot in a manner which may be
# significant for xG

### Replace Missing Values of Numerical Features

In [56]:
# Fill no pass shot numerical feature values with 0

extracted_data['pass_length'].fillna(0,
                                     inplace = True)

extracted_data['pass_angle'].fillna(0,
                                    inplace = True)

### Replace Values of Categorical Features No Pass Events

In [57]:
# Compare no pass shots v pass_type values

extracted_data.loc[extracted_data['pass_length'] == 0]['pass_type'].value_counts(dropna = False)

NaN    1943
Name: pass_type, dtype: int64

In [58]:
# Compare no pass shots v pass_technique values

extracted_data.loc[extracted_data['pass_length'] == 0]['pass_technique'].value_counts(dropna = False)

NaN    1943
Name: pass_technique, dtype: int64

In [59]:
# Fill no pass shot categorical feature values with 'No Pass'

# Note: will not replace for all missing values, just no pass events

extracted_data.loc[extracted_data['pass_length'] == 0,
                   ['pass_height',
                    'pass_type',
                    'pass_technique']] = 'No Pass'

### Assess Values of Boolean Features No Pass Events

In [60]:
# Define boolean_pass_features

boolean_pass_features = ['pass_switch',
                         'pass_through_ball',
                         'pass_backheel',
                         'pass_cross',
                         'pass_cut_back',
                         'pass_inswinging',
                         'pass_straight',
                         'pass_outswinging']

In [61]:
# Compare shot events with no pass v boolean_pass_features values

extracted_data.loc[extracted_data['pass_length'] == 0][boolean_pass_features].value_counts()

pass_switch  pass_through_ball  pass_backheel  pass_cross  pass_cut_back  pass_inswinging  pass_straight  pass_outswinging
False        False              False          False       False          False            False          False               1943
dtype: int64

In [62]:
# All no pass pass-related boolean feature values are currently 'False'

# No filling required

## Additional Missing Values

In [63]:
extracted_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 6079
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   period_x              6080 non-null   int64         
 1   timestamp_x           6080 non-null   datetime64[ns]
 2   play_pattern_x        6080 non-null   object        
 3   location_x            6080 non-null   object        
 4   under_pressure_x      6080 non-null   bool          
 5   shot_statsbomb_xg     6080 non-null   float64       
 6   shot_end_location     6080 non-null   object        
 7   shot_technique        6080 non-null   object        
 8   goal                  6080 non-null   bool          
 9   shot_type             6080 non-null   object        
 10  shot_body_part        6080 non-null   object        
 11  shot_one_on_one       6080 non-null   bool          
 12  shot_open_goal        6080 non-null   bool          
 13  shot_first_time   

### pass_type

In [64]:
extracted_data['pass_type'].value_counts(dropna = False)

NaN             3177
No Pass         1943
Corner           400
Recovery         305
Free Kick        201
Throw-in          42
Interception      10
Goal Kick          1
Kick Off           1
Name: pass_type, dtype: int64

In [65]:
# pass_type defined values are set-plays and defensive recoveries
# Assume missing values are from open play

# Fill pass_type missing values with 'Open Play'

extracted_data['pass_type'].fillna('Open Play',
                                   inplace = True)

### pass_technique

In [66]:
extracted_data['pass_technique'].value_counts(dropna = False)

NaN             3782
No Pass         1943
Through Ball     198
Inswinging        76
Outswinging       55
Straight          26
Name: pass_technique, dtype: int64

In [67]:
# pass_technique defined values are specialized passes
# Assume missing values are standard passes

# Fill pass_technique missing values with 'Standard'

extracted_data['pass_technique'].fillna('Standard',
                                        inplace = True)

## Results

In [67]:
extracted_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6080 entries, 0 to 6079
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   period_x              6080 non-null   int64         
 1   timestamp_x           6080 non-null   datetime64[ns]
 2   play_pattern_x        6080 non-null   object        
 3   location_x            6080 non-null   object        
 4   under_pressure_x      6080 non-null   bool          
 5   shot_statsbomb_xg     6080 non-null   float64       
 6   shot_end_location     6080 non-null   object        
 7   shot_technique        6080 non-null   object        
 8   goal                  6080 non-null   bool          
 9   shot_type             6080 non-null   object        
 10  shot_body_part        6080 non-null   object        
 11  shot_one_on_one       6080 non-null   bool          
 12  shot_open_goal        6080 non-null   bool          
 13  shot_first_time   

# Split Location Coordinates Features

In [68]:
extracted_data['location_x'].head()

0    [109.0, 46.0]
1    [113.0, 35.0]
2     [94.0, 43.0]
3     [86.0, 34.0]
4     [94.0, 33.0]
Name: location_x, dtype: object

In [69]:
extracted_data['shot_end_location'].head()

0         [112.0, 45.0]
1    [120.0, 32.9, 0.4]
2    [120.0, 42.8, 0.5]
3    [119.0, 33.3, 0.5]
4    [120.0, 34.8, 0.5]
Name: shot_end_location, dtype: object

### location_x

In [ ]:
# Split shot location coordinates, location_x, into separate x and y-coordinates

shot_location_df = pd.DataFrame(extracted_data['location_x'].tolist(),
                                index = extracted_data.index)

In [ ]:
# Replace location_x with shot_location_x and shot_location_y

extracted_data.drop('location_x',
                    axis = 1,
                    inplace = True)

extracted_data['shot_location_y'] = shot_location_df[0]
extracted_data['shot_location_x'] = shot_location_df[1]

### shot_end_location

In [ ]:
# Split shot end location coordinates, shot_end_location_x,
# into separate x, y, and z-coordinates

end_location_df = pd.DataFrame(extracted_data['shot_end_location'].tolist(),
                               index = extracted_data.index)

In [ ]:
# Drop y-coordinate
# All shots are aimed to end at the endline (120)

In [ ]:
print('z-coordinate NA:',
      end_location_df[2].isna().sum(),
      '\n'
      'z-coordinate percent NA:',
      ((end_location_df[2].isna().sum()) / (extracted_data.shape[0]) * 100))

z-coordinate NA: 1796 
z-coordinate percent NA: 29.539473684210527


In [ ]:
# Drop z-coordinate

In [ ]:
# Replace shot_end_location with x-coordinate

extracted_data['shot_end_location'] = end_location_df[1]

# Outliers

## Numeric Features

In [ ]:
# Function defining outliers

def iqr_outliers(feature):
  q1 = extracted_data[feature].quantile(0.25)
  q3 = extracted_data[feature].quantile(0.75)

  iqr = q3 - q1

  lower_limit = q1 - (1.5 * iqr)
  upper_limit = q3 + (1.5 * iqr)

  return lower_limit, upper_limit

### shot_location_x

In [ ]:
# Vizualize distribution of shot_location_x

fig = px.box(extracted_data,
             y = 'shot_location_x')

fig.show()

In [ ]:
lower_q_shot_location_x, upper_q_shot_location_x = iqr_outliers('shot_location_x')

print('shot_location_x Outliers:',
      '\n',
      'Wider than',
      (round((40 - lower_q_shot_location_x),
       2)),
      'Yards Left of Center',
      '\n',
      'Wider than',
      (round((upper_q_shot_location_x - 40),
       2)),
      'Yards Right of Center')

shot_location_x Outliers: 
 Wider than 28.15 Yards Left of Center 
 Wider than 28.25 Yards Right of Center


In [ ]:
outliers_shot_location_x = extracted_data[(extracted_data['shot_location_x'] <
                                           lower_q_shot_location_x) |
                                          (extracted_data['shot_location_x'] >
                                           upper_q_shot_location_x)]['shot_location_x']

print('shot_location_x Outlier Count:',
      len(outliers_shot_location_x))

shot_location_x Outlier Count: 16


In [ ]:
outliers_shot_location_x

424     69.0
1205    69.0
1250    75.0
1398    11.0
1543     5.0
1902     7.0
2429    78.6
2492    11.8
3083     0.7
3669    75.6
3715    75.0
4286    11.5
4317    10.1
4399    76.5
4687    79.4
5752    71.0
Name: shot_location_x, dtype: float64

In [ ]:
# Drop outliers

extracted_data.drop(outliers_shot_location_x.index,
                    inplace = True)

In [ ]:
extracted_data['shot_location_x'].describe()

count    6064.000000
mean       40.287385
std         9.799719
min        12.400000
25%        33.000000
50%        40.100000
75%        47.000000
max        68.200000
Name: shot_location_x, dtype: float64

In [ ]:
len(extracted_data)

6064

### shot_location_y

In [ ]:
# Vizualize distribution of shot_location_y

fig = px.box(extracted_data,
             y = 'shot_location_y')

fig.show()

In [ ]:
# Note: Only shots significantly further from goal v the median will be considered outliers
# Shots closer to goal are more desireable and will likely have higher xG

lower_q_shot_location_y, upper_q_shot_location_y = iqr_outliers('shot_location_y')

print('shot_location_y Outliers:',
      '\n',
      'Further than',
      (round((120 - lower_q_shot_location_y),
       2)),
      'Yards from Goal')

shot_location_y Outliers: 
 Further than 43.0 Yards from Goal


In [ ]:
outliers_shot_location_y = extracted_data[(extracted_data['shot_location_y'] <
                                           lower_q_shot_location_y)]['shot_location_y']

print('shot_location_y Outlier Count:',
      len(outliers_shot_location_y))

shot_location_y Outlier Count: 21


In [ ]:
outliers_shot_location_y

945     76.0
1186    71.0
1323    63.0
1339    58.0
1370    76.0
2050    76.0
2420    74.9
2758    64.0
2769    68.0
3450    68.2
3753    72.8
4080    75.2
4234    74.7
4276    71.3
4470    74.9
4698    70.4
5456    77.0
5504    61.0
5549    76.0
5998    77.0
6024    76.0
Name: shot_location_y, dtype: float64

In [ ]:
# Drop outliers

extracted_data.drop(outliers_shot_location_y.index,
                    inplace = True)

In [ ]:
extracted_data['shot_location_y'].describe()

count    6043.000000
mean      104.003690
std         8.819068
min        77.800000
25%        97.700000
50%       105.600000
75%       111.000000
max       120.000000
Name: shot_location_y, dtype: float64

### shot_end_location

In [ ]:
# Vizualize distribution of shot_end_location

fig = px.box(extracted_data,
             y = 'shot_end_location')

fig.show()

In [ ]:
lower_q_shot_end_location, upper_q_shot_end_location = iqr_outliers('shot_end_location')

print('shot_location_x Outliers:',
      '\n',
      'Wider than',
      (round((40 - lower_q_shot_end_location),
       2)),
      'Yards Left of Center',
      '\n',
      'Wider than',
      (round((upper_q_shot_end_location - 40),
       2)),
      'Yards Right of Center')

shot_location_x Outliers: 
 Wider than 14.7 Yards Left of Center 
 Wider than 14.9 Yards Right of Center


In [ ]:
outliers_shot_end_location = extracted_data[(extracted_data['shot_end_location'] <
                                           lower_q_shot_end_location) |
                                          (extracted_data['shot_end_location'] >
                                           upper_q_shot_end_location)]['shot_end_location']

print('shot_end_location Outlier Count:',
      len(outliers_shot_end_location))

shot_end_location Outlier Count: 198


In [ ]:
# Retain outliers

# Poor shot end location does not indicate a poor shot opportunity

### pass_length

In [ ]:
# Vizualize distribution of pass_length

fig = px.box(extracted_data,
             y = 'pass_length')

fig.show()

In [ ]:
lower_q_pass_length, upper_q_pass_length = iqr_outliers('pass_length')

print('pass_length Outliers:',
      '\n',
      'Further than',
      (round((upper_q_pass_length),
       2)),
      'Yards')

pass_length Outliers: 
 Further than 69.58 Yards


In [ ]:
outliers_pass_length = extracted_data[(extracted_data['pass_length'] >
                                       upper_q_pass_length)]['pass_length']

print('pass_length Outlier Count:',
      len(outliers_pass_length))

pass_length Outlier Count: 29


In [ ]:
# Retain outliers
# Long passes can create high quality shot opportunities

### pass_angle

In [ ]:
# Vizualize distribution of pass_angle

fig = px.box(extracted_data,
             y = 'pass_angle')

fig.show()

In [ ]:
lower_q_pass_angle, upper_q_pass_angle = iqr_outliers('pass_angle')

print('pass_angle Outliers:',
      '\n',
      'Wider than',
      (round((45 - lower_q_shot_location_x),
       2)),
      'Degrees Left of Center',
      '\n',
      'Wider than',
      (round((upper_q_shot_location_x - 45),
       2)),
      'Degrees Right of Center')

pass_angle Outliers: 
 Wider than 33.15 Degrees Left of Center 
 Wider than 23.25 Degrees Right of Center


In [ ]:
outliers_pass_angle = extracted_data[(extracted_data['pass_angle'] <
                                      lower_q_pass_angle) |
                                          (extracted_data['pass_angle'] >
                                           upper_q_pass_angle)]['pass_angle']

print('pass_angle Outlier Count:',
      len(outliers_pass_angle))

pass_angle Outlier Count: 64


In [ ]:
# Retain outliers
# Narrow angle passes can create high quality shot opportunities

## Total Dropped

In [ ]:
print('Total Dropped Outliers:',
      (len(outliers_shot_location_x) +
       len(outliers_shot_location_y)),
      '\n',
      'Percent Dropped Outliers:',
      (round(((len(outliers_shot_location_x) +
       len(outliers_shot_location_y)) / 
       len(extracted_data)),
       3)),
      '%')

Total Dropped Outliers: 37 
 Percent Dropped Outliers: 0.006 %


# Combine Redundant Features

## shot_technique

In [ ]:
extracted_data['shot_technique'].value_counts()

Normal           5097
Half Volley       506
Volley            336
Lob                55
Backheel           20
Diving Header      15
Overhead Kick      14
Name: shot_technique, dtype: int64

In [ ]:
extracted_data['shot_redirect'].value_counts()

False    6021
True       22
Name: shot_redirect, dtype: int64

In [ ]:
# Compare shot_redirect values v shot_technique values

extracted_data.loc[extracted_data['shot_redirect']]['shot_technique'].value_counts()

Normal         13
Volley          6
Backheel        2
Half Volley     1
Name: shot_technique, dtype: int64

In [ ]:
# No changes

# shot_redirect is a unique descriptor from shot_technique values

## play_pattern_x

In [ ]:
extracted_data['play_pattern_x'].value_counts()

Regular Play      2166
From Throw In     1207
From Corner       1006
From Free Kick     912
From Counter       335
From Goal Kick     214
Other               70
From Keeper         68
From Kick Off       65
Name: play_pattern_x, dtype: int64

In [ ]:
extracted_data['shot_type'].value_counts()

Open Play    5804
Free Kick     186
Penalty        53
Name: shot_type, dtype: int64

In [ ]:
# Compare shot_type 'Free Kick' values v play_pattern_x values

extracted_data.loc[extracted_data['shot_type'] == 'Free Kick']['play_pattern_x'].value_counts()

From Free Kick    186
Name: play_pattern_x, dtype: int64

In [ ]:
# Combine shot_type 'Free Kick' values into play_pattern_x as 'Direct Free Kick'

# Assume differentiation between 191 shot_type 'Free Kick' values and 918
# play_pattern_x 'From Free Kick' values is if the free kick was a direct
# shot or a pass leading to a shot

extracted_data.loc[extracted_data['shot_type'] == 'Free Kick',
                   'play_pattern_x'] = 'Direct Free Kick'

In [ ]:
# Compare shot_type 'Penalty' values v play_pattern_x values

extracted_data.loc[extracted_data['shot_type'] == 'Penalty']['play_pattern_x'].value_counts()

Other    53
Name: play_pattern_x, dtype: int64

In [ ]:
# Combine shot_type 'Penalty' values into play_pattern_x

extracted_data.loc[extracted_data['shot_type'] == 'Penalty',
                   'play_pattern_x'] = 'Penalty'

In [ ]:
# Compare shot_type 'Corner' values v play_pattern_x values

extracted_data.loc[extracted_data['shot_type'] == 'Corner']['play_pattern_x'].value_counts()

Series([], Name: play_pattern_x, dtype: int64)

In [ ]:
# shot_type 'Corner' values match play_pattern_x 'From Corner' values

In [ ]:
# Drop shot_type

extracted_data.drop('shot_type',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_type'].value_counts()

Open Play       3166
No Pass         1924
Corner           400
Recovery         299
Free Kick        201
Throw-in          41
Interception      10
Kick Off           1
Goal Kick          1
Name: pass_type, dtype: int64

In [ ]:
# Compare pass_type 'Corner' values v play_pattern_x values

extracted_data.loc[extracted_data['pass_type'] == 'Corner']['play_pattern_x'].value_counts()

From Corner    400
Name: play_pattern_x, dtype: int64

In [ ]:
# pass_type 'Corner' values match play_pattern_x 'From Corner' values

In [ ]:
# Compare pass_type 'Recovery' values v play_pattern_x values

extracted_data.loc[extracted_data['pass_type'] == 'Recovery']['play_pattern_x'].value_counts()

Regular Play      113
From Throw In      50
From Corner        44
From Counter       39
From Free Kick     33
From Goal Kick     10
From Keeper         7
From Kick Off       2
Other               1
Name: play_pattern_x, dtype: int64

In [ ]:
# Compare pass_type 'Interception' values v play_pattern_x values

extracted_data.loc[extracted_data['pass_type'] == 'Interception']['play_pattern_x'].value_counts()

Regular Play      5
From Throw In     2
From Counter      2
From Goal Kick    1
Name: play_pattern_x, dtype: int64

In [ ]:
# Combine pass_type 'Recovery' and 'Interception' values into play_pattern_x as 'Turnover'

# Assume pass_type 'Recovery' values are events in which the ball was recovered
# from the opposition's play described by play_pattern_x values

# Combine pass_type 'Interception' values with 'Recovery' values as interceptions are
# a type of recovery

extracted_data.loc[extracted_data['pass_type'] == 'Recovery',
                   'play_pattern_x'] = 'Recovery'

In [ ]:
# Compare pass_type 'Throw-in' values v play_pattern_x values

extracted_data.loc[extracted_data['pass_type'] == 'Throw-in']['play_pattern_x'].value_counts()

From Throw In    41
Name: play_pattern_x, dtype: int64

In [ ]:
# pass_type 'Throw-in' values match play_pattern_x 'From Throw In' values

In [ ]:
# Compare pass_type 'Kick Off' values v play_pattern_x values

extracted_data.loc[extracted_data['pass_type'] == 'Kick Off']['play_pattern_x'].value_counts()

From Kick Off    1
Name: play_pattern_x, dtype: int64

In [ ]:
# pass_type 'Kick Off' values match play_pattern_x 'From Kick Off' values

In [ ]:
# Compare pass_type 'Goal Kick' values v play_pattern_x values

extracted_data.loc[extracted_data['pass_type'] == 'Goal Kick']['play_pattern_x'].value_counts()

From Goal Kick    1
Name: play_pattern_x, dtype: int64

In [ ]:
# pass_type 'Goal Kick' values match play_pattern_x 'From Goal Kick' values

In [ ]:
# Drop pass_type

extracted_data.drop('pass_type',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['counterpress'].value_counts()

False    6038
True        5
Name: counterpress, dtype: int64

In [ ]:
# Compare counterpress values v play_pattern_x values

extracted_data.loc[extracted_data['counterpress']]['play_pattern_x'].value_counts()

From Counter      2
Regular Play      2
From Goal Kick    1
Name: play_pattern_x, dtype: int64

In [ ]:
# Combine counterpress into play_pattern_x

extracted_data.loc[extracted_data['counterpress'] == True,
                   'play_pattern_x'] = 'From Counterpress'

In [ ]:
# Drop counterpress

extracted_data.drop('counterpress',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['shot_follows_dribble'].value_counts()

False    6040
True        3
Name: shot_follows_dribble, dtype: int64

In [ ]:
# Compare shot_follows_dribble values v play_pattern_x values

extracted_data.loc[extracted_data['shot_follows_dribble']]['play_pattern_x'].value_counts()

Direct Free Kick    1
Recovery            1
Regular Play        1
Name: play_pattern_x, dtype: int64

In [ ]:
# Combine shot_follows_dribble into play_pattern_x

extracted_data.loc[extracted_data['shot_follows_dribble'] == True,
                   'play_pattern_x'] = 'From Dribble'

In [ ]:
# Drop shot_follows_dribble

extracted_data.drop('shot_follows_dribble',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['play_pattern_x'].value_counts()

Regular Play         2050
From Throw In        1157
From Corner           962
From Free Kick        693
Recovery              298
From Counter          294
From Goal Kick        203
Direct Free Kick      185
From Kick Off          63
From Keeper            61
Penalty                53
Other                  16
From Counterpress       5
From Dribble            3
Name: play_pattern_x, dtype: int64

## pass_technique

In [ ]:
extracted_data['pass_technique'].value_counts()

Standard        3764
No Pass         1924
Through Ball     198
Inswinging        76
Outswinging       55
Straight          26
Name: pass_technique, dtype: int64

In [ ]:
extracted_data['pass_through_ball'].value_counts()

False    5845
True      198
Name: pass_through_ball, dtype: int64

In [ ]:
# Compare pass_through_ball values v pass_technique

extracted_data.loc[extracted_data['pass_through_ball']]['pass_technique'].value_counts()

Through Ball    198
Name: pass_technique, dtype: int64

In [ ]:
# Drop pass_through_ball

# pass_through_ball True values match pass_technique 'Through Ball' values

extracted_data.drop('pass_through_ball',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_inswinging'].value_counts()

False    5967
True       76
Name: pass_inswinging, dtype: int64

In [ ]:
# Compare pass_inswinging values v pass_technique

extracted_data.loc[extracted_data['pass_inswinging']]['pass_technique'].value_counts()

Inswinging    76
Name: pass_technique, dtype: int64

In [ ]:
# Drop pass_inswinging

# pass_inswinging True values match pass_technique 'Inswinging' values

extracted_data.drop('pass_inswinging',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_outswinging'].value_counts()

False    5988
True       55
Name: pass_outswinging, dtype: int64

In [ ]:
# Compare pass_outswinging values v pass_technique

extracted_data.loc[extracted_data['pass_outswinging']]['pass_technique'].value_counts()

Outswinging    55
Name: pass_technique, dtype: int64

In [ ]:
# Drop pass_outswinging

# pass_outswinging True values match pass_technique 'Outswinging' values

extracted_data.drop('pass_outswinging',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_straight'].value_counts()

False    6017
True       26
Name: pass_straight, dtype: int64

In [ ]:
# Compare pass_straight values v pass_technique

extracted_data.loc[extracted_data['pass_straight']]['pass_technique'].value_counts()

Straight    26
Name: pass_technique, dtype: int64

In [ ]:
# Drop pass_straight

# pass_straight True values match pass_technique 'Straight' values

extracted_data.drop('pass_straight',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_cross'].value_counts()

False    5289
True      754
Name: pass_cross, dtype: int64

In [ ]:
# Compare pass_cross values v pass_technique

extracted_data.loc[extracted_data['pass_cross']]['pass_technique'].value_counts()

Standard        751
Through Ball      3
Name: pass_technique, dtype: int64

In [ ]:
# Combine pass_cross into pass_technique

extracted_data.loc[extracted_data['pass_cross'] == True,
                   'pass_technique'] = 'Cross'

In [ ]:
# Drop pass_cross

extracted_data.drop('pass_cross',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_backheel'].value_counts()

False    6030
True       13
Name: pass_backheel, dtype: int64

In [ ]:
# Compare pass_backheel values v pass_technique

extracted_data.loc[extracted_data['pass_backheel']]['pass_technique'].value_counts()

Standard    13
Name: pass_technique, dtype: int64

In [ ]:
# Combine pass_backheel into pass_technique

extracted_data.loc[extracted_data['pass_backheel'] == True,
                   'pass_technique'] = 'Backheel'

In [ ]:
# Drop pass_backheel

extracted_data.drop('pass_backheel',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_cut_back'].value_counts()

False    5935
True      108
Name: pass_cut_back, dtype: int64

In [ ]:
# Compare pass_cut_back values v pass_technique

extracted_data.loc[extracted_data['pass_cut_back']]['pass_technique'].value_counts()

Cross       63
Standard    43
Backheel     2
Name: pass_technique, dtype: int64

In [ ]:
# Combine pass_cut_back into pass_technique for values of 'Standard'

extracted_data.loc[(extracted_data['pass_cut_back'] == True) &
                   (extracted_data['pass_technique'] == 'Standard'),
                   'pass_technique'] = 'Cut Back'

In [ ]:
# Drop pass_cut_back

extracted_data.drop('pass_cut_back',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_switch'].value_counts()

False    5720
True      323
Name: pass_switch, dtype: int64

In [ ]:
# Compare pass_switch values v pass_technique

extracted_data.loc[extracted_data['pass_switch']]['pass_technique'].value_counts()

Standard        182
Cross            46
Inswinging       41
Outswinging      39
Straight         13
Through Ball      2
Name: pass_technique, dtype: int64

In [ ]:
# Combine pass_switch into pass_technique for values of 'Standard'

extracted_data.loc[(extracted_data['pass_switch'] == True) &
                   (extracted_data['pass_technique'] == 'Standard'),
                   'pass_technique'] = 'Switch'

In [ ]:
# Drop pass_switch

extracted_data.drop('pass_switch',
                    axis = 1,
                    inplace = True)

In [ ]:
extracted_data['pass_technique'].value_counts()

Standard        2775
No Pass         1924
Cross            754
Through Ball     195
Switch           182
Inswinging        76
Outswinging       55
Cut Back          43
Straight          26
Backheel          13
Name: pass_technique, dtype: int64

In [ ]:
# Combine pass_technique values 'Switch', 'Inswinging', 'Outswinging', and
# 'Straight' into value 'Cross'

# Values are types of crosses
# Value 'Cross' is nonspecific, but the most frequent
# Combining into single, less specific, value could increase importance in modeling

extracted_data.loc[extracted_data['pass_technique'].isin(['Switch',
                                                          'Inswinging',
                                                          'Outswinging',
                                                          'Straight']),
                   'pass_technique'] = 'Cross'

In [ ]:
extracted_data['pass_technique'].value_counts()

Standard        2775
No Pass         1924
Cross           1093
Through Ball     195
Cut Back          43
Backheel          13
Name: pass_technique, dtype: int64

# Update Feature Names

In [ ]:
extracted_data.rename(columns = {'period_x' : 'period',
                                 'timestamp_x' : 'timestamp',
                                 'play_pattern_x' : 'play_pattern',
                                 'under_pressure_x' : 'under_pressure',
                                 'shot_statsbomb_xg' : 'statsbomb_xg',
                                 'shot_end_location' : 'end_location',
                                 'shot_one_on_one': 'one_on_one',
                                 'shot_open_goal' : 'open_goal',
                                 'shot_first_time' : 'first_time',
                                 'shot_redirect' : 'redirected',
                                 'shot_deflected' : 'deflected',
                                 'shot_location_y' : 'location_y',
                                 'shot_location_x' : 'location_x',
                                 'pass_technique' : 'pass_type'},
                      inplace = True)

# Update Feature Order

In [ ]:
extracted_data = extracted_data[['goal',
                                 'statsbomb_xg',
                                 'period',
                                 'timestamp',
                                 'location_x',
                                 'location_y',
                                 'end_location',
                                 'shot_technique',
                                 'shot_body_part',
                                 'play_pattern',
                                 'pass_length',
                                 'pass_angle',
                                 'pass_height',
                                 'pass_type',
                                 'under_pressure',
                                 'one_on_one',
                                 'open_goal',
                                 'first_time',
                                 'redirected',
                                 'deflected']]

# ProfileReport 2

In [ ]:
pr_report = pr(extracted_data)
pr_report

Output hidden; open in https://colab.research.google.com to view.

# Cleaned Data

In [ ]:
cleaned_data = extracted_data

In [ ]:
cleaned_data.head()

,goal,statsbomb_xg,period,timestamp,location_x,location_y,end_location,shot_technique,shot_body_part,play_pattern,pass_length,pass_angle,pass_height,pass_type,under_pressure,one_on_one,open_goal,first_time,redirected,deflected
0,False,0.266154,1,2021-10-08 00:04:38.609,46.0,109.0,45.0,Normal,Left Foot,Regular Play,11.180340,0.463648,Ground Pass,Standard,True,False,False,False,False,False
1,False,0.093521,1,2021-10-08 00:11:45.046,35.0,113.0,32.9,Normal,Head,From Free Kick,37.735924,-0.558599,High Pass,Standard,True,True,False,False,False,False
2,False,0.036171,1,2021-10-08 00:18:03.461,43.0,94.0,42.8,Normal,Left Foot,Regular Play,11.180340,-2.034444,Ground Pass,Standard,True,False,False,False,False,False
3,False,0.016625,1,2021-10-08 00:23:11.935,34.0,86.0,33.3,Normal,Left Foot,From Goal Kick,13.892444,2.098871,Ground Pass,Standard,True,False,False,False,False,False
4,False,0.030716,1,2021-10-08 00:23:45.810,33.0,94.0,34.8,Normal,Right Foot,From Goal Kick,14.560220,1.292497,Ground Pass,Standard,False,False,False,False,False,False


In [ ]:
print('Total Events:',
      len(cleaned_data))

Total Events: 6043


In [ ]:
print('Total Features:',
      cleaned_data.shape[1])

Total Features: 20


In [ ]:
# Save cleaned_data

cleaned_data.to_parquet('/content/drive/MyDrive/expected_goals/data_cleaning/dataframes/cleaned_data.parquet')

In [ ]:
print('extracted_data Filesize:',
      path('/content/drive/MyDrive/expected_goals/data_cleaning/dataframes/cleaned_data.parquet').stat().st_size,
      'bytes')

extracted_data Filesize: 248621 bytes


Continued in [expected_goals_feature engineering_notebook]()

*3 of 7*